In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series, Timestamp
import tensorflow as tf
import tensorflow.compat.v1 as tf1
from tensorflow.compat.v1 import Session, ConfigProto
from tensorflow.python.eager.context import PhysicalDevice
from typing import Dict, List, Union, Generator, Tuple
import os
from numpy import load
import time
from tensorflow.python.keras.engine.functional import Functional

In [2]:
import sys
sys.path.insert(0, '..')

In [3]:
from data_formatters.base import GenericDataFormatter, InputTypes, DataTypes
from data_formatters.sorgenia_wind import SorgeniaFormatter

In [4]:
from expt_settings.configs import ExperimentConfig
from libs.hyperparam_opt import HyperparamOptManager
from libs.tft_model import TemporalFusionTransformer
import libs.utils as utils

In [9]:
class MyPredictor(object):
    def __init__(self, model: TemporalFusionTransformer):
        self.model = model
            
    def load(self, tf_session: Session, model_folder: str, cp_name: str, scope: str = None, verbose: bool = False):
        """Loads Tensorflow graph from checkpoint.
          Args:
            tf_session: Session to load graph into
            model_folder: Folder containing serialised model
            cp_name: Name of Tensorflow checkpoint
            scope: Variable scope to use.
            verbose: Whether to print additional debugging information.
         """
        # Load model proper
        load_path: str = os.path.join(model_folder, '{0}.ckpt'.format(cp_name))

        print('Loading model from {0}'.format(load_path))

        initial_vars = set(
            [v.name for v in tf1.get_default_graph().as_graph_def().node])

        # Saver
        if scope is None:
            saver = tf1.train.Saver()
        else:
            var_list = tf1.get_collection(tf1.GraphKeys.GLOBAL_VARIABLES, scope=scope)
            saver = tf1.train.Saver(var_list=var_list, max_to_keep=100000)
        # Load
        saver.restore(tf_session, load_path)
        all_vars = set([v.name for v in tf1.get_default_graph().as_graph_def().node])

        if verbose:
            print('Restored {0}'.format(','.join(initial_vars.difference(all_vars))))
            print('Existing {0}'.format(','.join(all_vars.difference(initial_vars))))
            print('All {0}'.format(','.join(all_vars)))

        print('Done.')
        
    def predict(self, instances: DataFrame, **kwargs) -> List[Tuple]:
        output_map: Dict = self.model.predict(instances, return_targets=False)
        output: DataFrame = self.formatter.format_predictions(output_map.get("p50"))
        # convert output to a list if that's required by GCP
        return [tuple(x) for x in output.to_numpy()]
        
        
    @classmethod
    def from_path(cls, model_dir: str):
        """
         :params : folder with model checkpoints and params
        """
        if tf.test.gpu_device_name(): 
            print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
        else:
            print("Please install GPU version of TF")
        gpu: List[PhysicalDevice] = tf.config.experimental.list_physical_devices('GPU')
        tf.config.experimental.set_memory_growth(gpu[0], True)
        tf_config: ConfigProto = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)
        config = ExperimentConfig('sorgenia_wind', model_dir)
        formatter = config.make_data_formatter()
        # Sets up default params
        fixed_params: Dict = formatter.get_experiment_params()
        print("Fixed parameters are ", fixed_params)
        params: Dict = formatter.get_default_model_params()
        print("regular params are ", params)
        params["model_folder"]: str = os.path.join(config.model_folder, "fixed")
        model_folder = os.path.join(config.model_folder, "fixed")
        # Sets up hyperparam manager
        print("*** Loading hyperparm manager ***")
        opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                           fixed_params, model_folder)
        tf1.reset_default_graph()
        with tf.Graph().as_default(), tf1.Session(config=tf_config) as sess:
            tf1.keras.backend.set_session(sess)
            params: Dict = opt_manager.get_next_parameters()
#             print(params)
            params['exp_name'] = 'sorgenia_wind'
            params['data_folder'] = config.data_csv_path
            model = TemporalFusionTransformer(params, use_cudnn=True)
            params.pop('exp_name', None)
            params.pop('data_folder', None)
            model.load(tf.compat.v1.keras.backend.get_session(),
                    model_folder, cp_name="TemporalFusionTransformer", scope="TemporalFusionTransformer")
        
        return cls(model)

In [10]:
predictor = MyPredictor.from_path(r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs')

Default GPU Device:/device:GPU:0
Selecting GPU ID=0
Fixed parameters are  {'total_time_steps': 180, 'num_encoder_steps': 168, 'num_epochs': 100, 'early_stopping_patience': 10, 'multiprocessing_workers': 5, 'column_definition': [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('time', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('kwh', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('dew_point_2m_C', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('temperature_2m_C', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('msl_pressure_hPa', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('sfc_pressure_hPa', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('precipitation_1h_mm', <DataTypes.REAL_VALUED: 0>, <Inp

JSONDecodeError: Expecting value: line 1 column 1 (char 0)